## Geodesic Distances

Using the [fdasrsf](https://fdasrsf-python.readthedocs.io/en/latest/) library, we compute the geodesic distances between pairs of open curves.

#### Imports / Functions

In [2]:
import pandas as pd
import numpy as np
from copy import deepcopy
import tqdm
from fdasrsf.geodesic import geod_sphere

In [34]:
def rescale_curve(x,y,v=3):
    # Code to scale a curve so that it lies between +-v/2 (default: 1.5) in both axes.
    p = max(max(y)-min(y),max(x)-min(x))
    q = p/v
    x_ = x/q
    y_ = y/q
    mp = min(x_)+(max(x_)-min(x_))/2
    x_ = x_-mp
    mp = min(y_)+(max(y_)-min(y_))/2
    y_ = y_-mp
    return list(x_),list(y_)

#### Process Data

In [36]:
pth = "C:\\Users\\arian\\Documents\\GitHub\\contourExtraction\\smoothedMussels_Scaled_251.csv" # Path to dataset.
table = pd.read_csv(pth)

### Pairwise Distances by Group

Here, we compute distance matrices per group.

In [52]:
all_groups = np.unique(table['type'])

In [ ]:
rescale = False # Set rescale = True, if you want to rescale curves to be within +-1.5 in the x,y axis.

for group in all_groups:
    filtered_table = table[table['type']==group]
    names = np.unique(filtered_table['id'])
    n = len(names)
    distances_DP = np.zeros((n,n))
    errors = []
    print("Computing distance matrix for group: "+str(group))
    for i in tqdm.tqdm(range(0,n)):
        x1 = list(filtered_table[filtered_table['id']==names[i]]['x'])
        y1 = list(filtered_table[filtered_table['id']==names[i]]['y'])
        if rescale == True:
            x1,y1 = rescale_curve(np.array(x1),np.array(y1))
        beta1 = np.column_stack([x1,y1]).T
        for j in range(i+1,n):
            x2 = list(filtered_table[filtered_table['id']==names[j]]['x'])
            y2 = list(filtered_table[filtered_table['id']==names[j]]['y'])
            if rescale == True:
                x2,y2 = rescale_curve(np.array(x2),np.array(y2))
            beta2 = np.column_stack([x2,y2]).T
            try:
                d,_,_, = geod_sphere(beta1, beta2)
            except:
                try:
                    d,_,_, = geod_sphere(beta2, beta1)
                except:
                    errors.append([i,j])
                    d = 100000

            distances_DP[i,j] = d
            distances_DP[j,i] = d
    dists_df = pd.DataFrame(distances_DP,index=names)
    dists_df.columns = names
    dists_df.index.name = 'id' 
    dists_df.to_csv('Open_SRVF_Distances_'+str(group)+'.csv')
    print("Computed distances between "+str(n)+" contours, with "+str(len(errors))+" errors.")
    # np.save('errors_'+str(group)+'.npy',errors) # Uncomment this if you'd wish to save the errors.
    

### Pairwise Distances on Entire Dataset

Here, we compute one distance matrix based on the entire dataset.

In [57]:
names = np.unique(table['id'])
n = len(names)

distances_DP = np.zeros((n,n))
errors = []

for i in tqdm.tqdm(range(0,n)):
    x1 = list(table[table['id']==names[i]]['x'])
    y1 = list(table[table['id']==names[i]]['y'])
    if rescale == True:
        x1,y1 = rescale_curve(np.array(x1),np.array(y1))
    beta1 = np.column_stack([x1,y1]).T
    for j in range(i+1,n):
        x2 = list(table[table['id']==names[j]]['x'])
        y2 = list(table[table['id']==names[j]]['y'])
        if rescale == True:
            x2,y2 = rescale_curve(np.array(x2),np.array(y2))
        beta2 = np.column_stack([x2,y2]).T
        try:
            d,_,_, = geod_sphere(beta1, beta2)
        except:
            try:
                d,_,_, = geod_sphere(beta2, beta1)
            except:
                errors.append([i,j])
                d = 100000

        distances_DP[i,j] = d
        distances_DP[j,i] = d
        
    pd.DataFrame(distances_DP).to_csv('Open_SRVF_Distances_ongoing.csv') # Save CSV distance matrix as you go along.
    # This is a precautionary step in case the program breaks, so that distances won't be deleted.
    # Comment out this step if you don't wish to save at each iteration.

dists_df = pd.DataFrame(distances_DP,index=names)
dists_df.columns = names
dists_df.index.name = 'id' 
dists_df.to_csv('Open_SRVF_Distances_All.csv')
print("Computed distances between "+str(n)+" contours, with "+str(len(errors))+" errors.")

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:12<00:00,  2.46s/it]

Computed distances between 5 contours, with 0 errors.
